In [1]:
import pyspark

In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
from pyspark.sql import SparkSession 

In [4]:
ss = SparkSession.builder.appName('practice').getOrCreate()

In [5]:
ss

In [13]:
df = ss.read.option('header','false').csv('C:/Users/Hitesharma.US/Downloads/PySpark_Assessment_Assignment_SparkSQL-DataSet/NYSE_daily.tsv',sep='\t')

In [14]:
df1 = ss.read.option('header','false').csv('C:/Users/Hitesharma.US/Downloads/PySpark_Assessment_Assignment_SparkSQL-DataSet/NYSE_dividends.tsv',sep='\t')

In [22]:
df.show(10)

+-------------+-----------+---------+---------+---------+--------+----------+-------+------------------+
|StockExchange|StockSymbol|     Date|OpenPrice|HighProce|LowPrice|ClosePrice| Volume|AdjustedClosePrice|
+-------------+-----------+---------+---------+---------+--------+----------+-------+------------------+
|         NYSE|        JEF| 2/8/2010|     25.4|    25.49|   24.78|     24.82|1134300|             24.82|
|         NYSE|        JEF| 2/5/2010|    24.91|    25.19|   24.08|     25.01|1765200|             25.01|
|         NYSE|        JEF| 2/4/2010|    26.01|     26.2|   24.85|     24.85|1414400|             24.85|
|         NYSE|        JEF| 2/3/2010|    26.23|    26.76|   26.22|     26.29|1066000|             26.29|
|         NYSE|        JEF| 2/2/2010|    26.08|    26.86|   25.78|     26.46|1496400|             26.46|
|         NYSE|        JEF| 2/1/2010|    25.61|    26.11|   25.36|     26.11|2381800|             26.11|
|         NYSE|        JEF|1/29/2010|    26.57|     26.

In [23]:
df1.show(10)

+-------------+-----------+----------+---------+
|StockExchange|StockSymbol|      Date|dividends|
+-------------+-----------+----------+---------+
|         NYSE|        JAH|12/30/2009|    0.075|
|         NYSE|        JAH| 9/29/2009|    0.075|
|         NYSE|        JGT|12/11/2009|    0.377|
|         NYSE|        JGT| 9/11/2009|    0.377|
|         NYSE|        JGT| 6/11/2009|    0.377|
|         NYSE|        JGT| 3/11/2009|    0.377|
|         NYSE|        JGT|12/11/2008|    0.377|
|         NYSE|        JGT| 9/11/2008|    0.451|
|         NYSE|        JGT| 6/11/2008|    0.451|
|         NYSE|        JGT| 3/12/2008|    0.451|
+-------------+-----------+----------+---------+
only showing top 10 rows



In [24]:
df.printSchema()

root
 |-- StockExchange: string (nullable = true)
 |-- StockSymbol: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- OpenPrice: string (nullable = true)
 |-- HighProce: string (nullable = true)
 |-- LowPrice: string (nullable = true)
 |-- ClosePrice: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- AdjustedClosePrice: string (nullable = true)



In [25]:
df1.printSchema()

root
 |-- StockExchange: string (nullable = true)
 |-- StockSymbol: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- dividends: string (nullable = true)



In [20]:
df = df.withColumnRenamed('_c0', 'StockExchange').withColumnRenamed('_c1', 'StockSymbol').withColumnRenamed('_c2', 'Date').withColumnRenamed('_c3', 'OpenPrice').withColumnRenamed('_c4', 'HighProce').withColumnRenamed('_c5', 'LowPrice').withColumnRenamed('_c6', 'ClosePrice').withColumnRenamed('_c7', 'Volume').withColumnRenamed('_c8', 'AdjustedClosePrice')

In [21]:
df1 = df1.withColumnRenamed('_c0', 'StockExchange').withColumnRenamed('_c1', 'StockSymbol').withColumnRenamed('_c2', 'Date').withColumnRenamed('_c3', 'dividends')

In [26]:
df.createOrReplaceTempView("vw_data")

# stock close price more than or equal to 200 and stock volume more than or equal to 10 million

In [31]:
ss.sql("select * from vw_data where ClosePrice>=200 and Volume >=10000000 ").show(10)

+-------------+-----------+----------+---------+---------+--------+----------+--------+------------------+
|StockExchange|StockSymbol|      Date|OpenPrice|HighProce|LowPrice|ClosePrice|  Volume|AdjustedClosePrice|
+-------------+-----------+----------+---------+---------+--------+----------+--------+------------------+
|         NYSE|       JNPR| 11/3/2000|      198|   216.88|  196.25|    216.13|13424800|            216.13|
|         NYSE|       JNPR|10/19/2000|   229.13|   234.31|     220|    232.58|11323800|            232.58|
|         NYSE|       JNPR|10/18/2000|   219.38|      235|   212.5|    213.88|15463100|            213.88|
|         NYSE|       JNPR|10/17/2000|   241.75|   241.81|     224|    229.19|16734200|            229.19|
|         NYSE|       JNPR|10/16/2000|   226.75|    244.5|     224|       243|17288400|               243|
|         NYSE|       JNPR|10/13/2000|   201.75|    229.5|  201.63|     228.5|19565000|             228.5|
|         NYSE|       JNPR|10/11/2000

In [29]:
df1.createOrReplaceTempView("vw1_data")

# List the companies that have given dividends more than 50 times. The list should include the number of times they have given dividends.

In [33]:
ss.sql("select count(dividends) count from vw1_data  group by StockSymbol having count>50").show(10)

+-----+
|count|
+-----+
|  114|
|   72|
|  104|
|   63|
|   58|
|   99|
|  160|
|   88|
|   91|
|   51|
+-----+
only showing top 10 rows



# List the companies along with their close price, dividends and the date when they gave dividends of more than0.01 when their daily close price was more than or equal to 100 sorted by dividends in ascending order.

In [36]:
ss.sql("select vw.StockSymbol, vw.ClosePrice,vw1.dividends,vw1.Date from vw_data vw join vw1_data vw1 on vw.Date=vw1.Date where vw1.dividends >0.01 and vw.ClosePrice >=100 order by vw1.dividends ").show(10)

+-----------+----------+---------+----------+
|StockSymbol|ClosePrice|dividends|      Date|
+-----------+----------+---------+----------+
|        JNJ|    104.62|    0.012| 8/25/1987|
|       JNPR|       197|   0.0125| 8/12/1999|
|       JNPR|    303.75|   0.0125|11/18/1999|
|        JPM|    144.19|   0.0125| 5/20/1998|
|       JNPR|    215.88|   0.0125| 2/11/2000|
|       JNPR|       148|   0.0125| 5/11/2000|
|        JNJ|    104.37|   0.0125|11/18/1999|
|       JNPR|    106.06|   0.0125|11/28/2000|
|        JPM|    119.81|   0.0125| 2/11/1998|
|        JNJ|    100.12|   0.0125|11/28/2000|
+-----------+----------+---------+----------+
only showing top 10 rows



In [39]:
ccp=df.join(df1,df.StockSymbol==df1.StockSymbol,"inner").select(df.StockSymbol,df.ClosePrice,df.Date,df1.dividends).filter((df1.dividends > 0.01)&(df.ClosePrice >= 100)).orderBy(df1.dividends)

In [40]:
ccp.show(10)

+-----------+----------+---------+---------+
|StockSymbol|ClosePrice|     Date|dividends|
+-----------+----------+---------+---------+
|        JNJ|     101.8|6/12/2001|  0.01042|
|        JNJ|     101.8|6/12/2001|  0.01042|
|        JNJ|    101.37|6/11/2001|  0.01042|
|        JNJ|    101.37|6/11/2001|  0.01042|
|        JNJ|    101.37|6/11/2001|  0.01042|
|        JNJ|    101.37|6/11/2001|  0.01042|
|        JNJ|     101.8|6/12/2001|  0.01042|
|        JNJ|    101.37|6/11/2001|  0.01042|
|        JNJ|    102.01| 6/8/2001|  0.01042|
|        JNJ|     101.8|6/12/2001|  0.01042|
+-----------+----------+---------+---------+
only showing top 10 rows



# Save the above lists as comma separated files.

In [42]:
a=ss.sql("select * from vw_data where ClosePrice>=200 and Volume >=10000000 ")

In [43]:
b=ss.sql("select count(dividends) count from vw1_data  group by StockSymbol having count>50")

In [44]:
c=ss.sql("select count(dividends) count from vw1_data  group by StockSymbol having count>50")

In [45]:
a.toPandas().to_csv('a.csv')

In [46]:
b.toPandas().to_csv('b.csv')

In [47]:
c.toPandas().to_csv('c.csv')